In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import re
from dotenv import load_dotenv
import pandas as pd

from IPython.display import Markdown, display, HTML
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
load_dotenv()
os.chdir(os.path.dirname(os.getcwd()))

df = pd.read_parquet("./data/splade_embeds.parquet")
# Convert column names to snake_case for compatibility with LanceDB
original_columns = df.columns
snake_case_columns = {col: re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in original_columns}
df.rename(columns=snake_case_columns, inplace=True)
df.head(2)

,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,state,kmeans_label,topic_title,splade_embeddings
0,1078,1575952538,https://www.reddit.com/r/legaladvice/comments/...,e8lsen,I applied for a job and after two interviews I...,"Failed a drug test due to amphetamines, I have...",employment,5,"[9.475638042064453e-05, 0.0005111666301983955,...",493,"""Validity of Schedule II Drug Prescription in ...",PR,8,Employment Legal Concerns and Issues,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2098,1577442453,https://www.reddit.com/r/legaladvice/comments/...,eg9ll2,"Hi everyone, thanks in advance for any guidanc...","Speeding ticket in Tennessee, Georgia Driver's...",driving,4,"[-0.006706413111028856, 0.020911016696181495, ...",252,"""Speeding ticket consequences for out-of-state...",KY,10,Legal Topics in Traffic Violations,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [3]:
df.sort_values(by='token_count', ascending=False, inplace=True)

In [4]:
test_query = df['body'].tolist()[6]
Markdown(test_query)

This will be a long post, but I really need some advice.

I live in Missouri and moved in with my boyfriend in 2005 after his dad moved out, so we've been on a month by month basis since then. We've always had problems, like things not being fixed or disruptive neighbors (screaming all hours of the night, cops and CPS, shared backyard taken over by boats, firepits, toys)  and we're pretty sure they've been in our apartment when we're not there. A long term (20+ years) tenant said they would do this, and other tenants who I was friendly with said they thought the same. I'm pretty sure my landlord stole a bong from me, since he was outside when we left and I hid it, and he and it was gone when we got back. We tore the place apart, but it's never turned up, and that was years ago. I've also thought numerous times that things in my drawers have been rearranged, and a few of my other neighbors over the years have said the same.

They have always been slow to fix things, if at all, and several things are broken: windows don't shut, bad wiring leading to lights not working (bathroom and second bedroom), one of the water knobs in the bathroom broke off so we use pliers to turn it, the oven, it goes on and on.

Over the summer we received an undated letter raising rent (they are dated when they would raise rent previously) and it also stated that ANY repairs done to the apartment would raise rent, and it specifically mentioned refrigerators and furnaces. Now, when my boyfriend moved in with his dad back in 2004, the landlord said he knew the furnace needed to be replaced, and was waiting for the furnace to go out before doing it. Three days after getting this letter, our furnace died, so we had no AC over the summer with my boyfriend prone to heat stroke since he's had it three times, and no heat this winter, although it hasn't been too bad with space heaters and electric blankets. 

Now the new problem is admittedly partially my fault. Rent is due on the first, but I was so busy (I work two jobs and they won't let me write checks since I wasn't on the original lease and my boyfriend doesn't have a bank account) that I didn't mail it until the second. I mailed it from the post office immediately after I purchased the money order. The landlady came by on the 10th pissed because she didn't have rent yet. Now, this has happened before; once it was in the mailbox when she got home and another time her husband deposited it in his account not hers, and another time he just forgot to tell her, so I figured it was something similar, or mail was just running slow since it's the beginning of December.

I showed her my money order stub, but she was adamant that I pay her. I went to the post office and paid to track the money order, which a refund could take a minimum of 60 days. I told her I was doing all of this and she agreed to it. She messaged me this morning saying that she still hasn't gotten the rent, and that I needed to make arrangements to pay her. I told her I was waiting on the post office, like she agreed to. I got online and tracked my request, and it said that "refund eligibility was being processed." I told her this, and she said regardless I was responsible for it, and I responded that I was waiting on the post office, and that if I had to pay dpuble and the end of this month I would.

Here's the thing: they've made no secret about the fact that they want us out; we're grandfathered in at a way cheaper rate than anyone else in the building, and they tell us this all the time. When we had a volatile neighbor who was on pills, calling the police on everyone and having them and CPS called on her, fights outside at all hours, screaming of the adults and children at all hours, pets they're not supposed to have that the landlords know about, nothing was done. The landlords would just complain that they wished the people would move, or suggest that we move, but would never do anything. These people finally moved away, and the landlords have just finished fixing the apartment. 

I believe that they have the rent and are acting like they don't, hoping I don't have the money to pay double rent and can kick us out. I have the money to pay December now, and January at the end of this month, but it will be tight. Should I just pay and wait for my refund? If the post office shows that my original money order was cashed, am I out that money? 

Please help; any and all advice is welcome. Thank you!

In [5]:
from src.search.query_expansion import segment

queries = segment(
    test_query,
    n='3 to 5'
)

2024-03-18 22:22:01 - INFO - Using pre-computed 'body' embeddings from existing column: splade_embeddings
2024-03-18 22:22:01 - INFO - Using pre-computed 'body' embeddings from existing column: splade_embeddings


In [6]:
queries.searches

[SubQuestion(chain_of_thought='To understand the legal rights and protections for tenants in Missouri, especially regarding landlord entry, repairs, and rent increases.', sub_question_topic='Tenant Rights in Missouri', sub_question_query='What are tenant rights in Missouri regarding landlord entry, repairs, and rent increases?', sub_question_keywords=['tenant rights', 'Missouri', 'landlord entry', 'repairs', 'rent increases']),
 SubQuestion(chain_of_thought='To explore the legal procedures and tenant options when a landlord is suspected of theft or unauthorized entry.', sub_question_topic='Dealing with Landlord Theft or Unauthorized Entry', sub_question_query='How can tenants in Missouri deal with suspected landlord theft or unauthorized entry?', sub_question_keywords=['landlord theft', 'unauthorized entry', 'tenant options', 'Missouri']),
 SubQuestion(chain_of_thought='To understand the process and implications of late rent payments, including the legal timeframe landlords must adhere

In [7]:
vector_results, keyword_results = queries.execute()



Thought: To understand the legal rights and protections for tenants in Missouri, especially regarding landlord entry, repairs, and rent increases.
Search topic: Tenant Rights in Missouri
Running vector (OpenAI) search on: What are tenant rights in Missouri regarding landlord entry, repairs, and rent increases?
Running keyword (SPLADE) search on: tenant rights, Missouri, landlord entry, repairs, rent increases
Returning 10 records from vector search and 10 from keywords
Results contain 14 unique IDs
---------------------------------------------------------------------------


Thought: To explore the legal procedures and tenant options when a landlord is suspected of theft or unauthorized entry.
Search topic: Dealing with Landlord Theft or Unauthorized Entry
Running vector (OpenAI) search on: How can tenants in Missouri deal with suspected landlord theft or unauthorized entry?
Running keyword (SPLADE) search on: landlord theft, unauthorized entry, tenant options, Missouri
Returning 10 

In [8]:
from src.search.doc_joiner import DocJoinerDF
from src.embedding_models.models import ColbertReranker
from src.search.rerank_openai import rerank_with_openai

In [9]:
contater = DocJoinerDF(join_mode="reciprocal_rank_fusion", top_k=20)

In [10]:
vector_res = contater.run(vector_results)
keyword_res = contater.run(keyword_results)
vector_res.shape, keyword_res.shape

((20, 17), (20, 17))

In [66]:
openai_rank_test = rerank_with_openai(keyword_results[0], query=test_query, text_col_name='body')
openai_rank_test.head()

,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,state,kmeans_label,topic_title,splade_embeddings,score,search_type,query,body_reranked,prediction,logprobs,probability,confidence,yes_probability
2744,3692,1589697300,https://www.reddit.com/r/legaladvice/comments/...,glapn0,I mainly want to know if I'm in my rights as a...,Tenants rights to receive notice before landlo...,housing,7,"[-0.004877123706320873, -0.006857454112966537,...",319,"""Understanding Tenant Rights: Can Landlords En...",PA,3,Rental Property and Landlord Matters,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16.978695,splade,"This will be a long post, but I really need so...",I mainly want to know if I'm in my rights as a...,No,-0.162727,0.849823,0.162727,0.150177
4275,4775,1590697989,https://www.reddit.com/r/legaladvice/comments/...,gse7e0,"Hello, I'm from Missouri! I've recently acqui...",Is my truck violating my lease? MO,housing,7,"[0.003063725374637794, -0.030413238548675448, ...",266,"""Is my medium duty truck violating my lease ag...",VA,3,Rental Property and Landlord Matters,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",13.275961,splade,"This will be a long post, but I really need so...","Hello, I'm from Missouri! I've recently acqui...",No,-0.067785,0.934461,0.067785,0.065539
4682,7572,1516064012,https://www.reddit.com/r/legaladvice/comments/...,7qok5z,Background: I currently have a lease through A...,[IL] Landlord selling condo unit halfway throu...,housing,7,"[-0.012060271207740768, -0.003256475704940407,...",297,"""Landlord selling condo: Can the realtor deman...",IN,3,Rental Property and Landlord Matters,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",15.413387,splade,"This will be a long post, but I really need so...",Background: I currently have a lease through A...,No,-0.054250,0.947195,0.054250,0.052805
4041,4830,1590685868,https://www.reddit.com/r/legaladvice/comments/...,gsa99b,I'm asking this on behalf of a very good frien...,Landlord/Tenant question for North Carolina,housing,7,"[-0.0006065469394306883, -0.023575009944144434...",127,"""Rent Increase and Tenant Protections: Underst...",MA,0,Legal Topics in Rental Properties,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",13.648916,splade,"This will be a long post, but I really need so...",I'm asking this on behalf of a very good frien...,No,-0.048355,0.952796,0.048355,0.047204
492,3749,1421803853,https://www.reddit.com/r/legaladvice/comments/...,2t4bww,"Hello Reddit,\n\nMy roommate and I are wanting...",Can someone explain what a rerent levy is?,housing,7,"[0.015440599531819121, -0.0001857048427183159,...",160,"""Breaking a lease in Kansas City, Missouri: Un...",NJ,0,Legal Topics in Rental Properties,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",12.767593,splade,"This will be a long post, but I really need so...","Hello Reddit,\n\nMy roommate and I are wanting...",No,-0.044801,0.956188,0.044801,0.043812


In [15]:
reranker = ColbertReranker(column='body')

In [16]:
vec_res = vector_res.drop(columns=['embeddings', 'splade_embeddings'])
kw_res = keyword_res.drop(columns=['embeddings', 'splade_embeddings'])

In [17]:
hybrid_results = reranker.rerank_hybrid(test_query, vec_res, kw_res)

In [20]:
hybrid_results.shape

(40, 15)

In [22]:
hybrid_results.drop_duplicates(subset=['body'], inplace=True)
hybrid_results.shape

(35, 15)

In [23]:
hybrid_results.head(20)['search_type'].value_counts()

search_type
splade    11
vector     9
Name: count, dtype: int64

In [26]:
from src.agent.tools.utils import aget_fact_patterns_df
import nest_asyncio
nest_asyncio.apply()

prep_df = await aget_fact_patterns_df(hybrid_results, 'body', "id")

Processing summaries: 100%|██████████| 35/35 [00:11<00:00,  2.96it/s]


In [27]:
prep_df.head(2)

,index,created_utc,full_link,id,body,title,text_label,flair_label,token_count,llm_title,state,kmeans_label,topic_title,search_type,_relevance_score,summary
0,5872,1576255862,https://www.reddit.com/r/legaladvice/comments/...,ea6gvy,"This will be a long post, but I really need so...",Who do I contact if I think my landlord is bei...,housing,7,1001,"""Legal issues with landlord: repairs, theft, r...",MH,3,Rental Property and Landlord Matters,splade,0.939139,A tenant in Missouri experiences multiple ongo...
1,9173,1422499769,https://www.reddit.com/r/legaladvice/comments/...,2u1eqk,This is a follow up post to [this post](http:/...,(Florida) Need to know how to dispute refusal ...,housing,7,665,"""Landlord withholding security deposit - How t...",ID,0,Legal Topics in Rental Properties,splade,0.870435,A renter details a series of problematic renta...


In [32]:
from src.agent.tools.utils import create_formatted_input, get_final_answer

formatted_input = create_formatted_input(
    prep_df, test_query, context_token_limit=8000
    )

response_model = get_final_answer(formatted_input, model_name="gpt-4-turbo-preview")

In [33]:
Markdown(response_model.research_report)

The new query presents a multifaceted legal issue involving poor maintenance, unresponsive landlords, potential unauthorized entry and theft by the landlord, and a rent payment dispute possibly being leveraged for eviction purposes. The tenant's situation in Missouri shares several parallels with past cases, providing an informative basis for comparison. 

The scenario with the tenant experiencing multiple ongoing issues, including unaddressed maintenance problems and unauthorized entry, closely mirrors the situation described in [Search Result 1](https://www.reddit.com/r/legaladvice/comments/ea6gvy/who_do_i_contact_if_i_think_my_landlord_is_being/). Both highlight the challenges faced by tenants dealing with landlords who are slow or refuse to make necessary repairs, coupled with the suspicion that the landlord is entering their premises without permission. The advice sought in both queries centers on how to handle these complex tenant-landlord relationship issues, especially regarding the landlord's responsibilities for upkeep and respect for tenant privacy. 

The challenge of dealing with a rent payment dispute, where the tenant has made a good faith effort to pay rent on time but faces difficulties due to mail delivery issues, aligns with the primary concern of [Search Result 1](https://www.reddit.com/r/legaladvice/comments/ea6gvy/who_do_i_contact_if_i_think_my_landlord_is_being/) and is further compounded by the landlord's threats and the inherent fear of eviction. This situation also resonates with concerns raised in [Search Result 22](https://www.reddit.com/r/legaladvice/comments/56uax5/missouri_landlord_attempting_to_evict/), where a Missouri resident faces eviction without proper notice and questionable rent increase demands. Both scenarios emphasize the precarious nature of tenant-landlord trust and the legal technicalities surrounding proper eviction processes and rent increase notices. 

Notably, the current query uniquely combines these elements into a single narrative, presenting a multifaceted legal dilemma. The past cases offer valuable insights into addressing each aspect individually but also illustrate the complexity of navigating these issues concurrently. Legal advice and intervention might be necessary to ensure that the tenant's rights are protected, especially when facing potential eviction under dubious circumstances. The incorporation of concerns regarding unauthorized entry, property theft, and exploitation of rent payment issues adds layers to the tenant's predicament, underscoring the importance of seeking comprehensive legal advice tailored to the specificities of the situation.

In [34]:
from src.search.query_expansion import generate_subquestions

questions = generate_subquestions(test_query, n='any number of')
questions.questions

['Should I just pay the rent and wait for my refund?',
 'If the post office shows that my original money order was cashed, am I out that money?',
 'What can I do about a landlord who is slow to make repairs?',
 'Is it legal for my landlord to enter my apartment without permission?',
 'Can a landlord raise rent in response to making repairs?',
 'How do I handle disruptive neighbors?',
 'What should I do if I suspect my landlord of stealing from me?',
 'What are my rights as a tenant in Missouri?',
 'Is it legal to use pliers to turn on water in lieu of a broken knob?',
 "What actions can I take if I've been treated unfairly by my landlord?"]

In [35]:
questions._raw_response.usage

CompletionUsage(completion_tokens=147, prompt_tokens=1130, total_tokens=1277)

In [36]:
from src.search.query_filter import generate_query_plan, auto_filter_fts_search

In [37]:
query_plan = generate_query_plan(
    input_df=df,
    query=questions.questions[0],
    filter_fields=[
        'state',
    ]
)
filtered_df = query_plan.filter_df(df=df)

2024-03-18 01:12:36 - INFO - Schema shown to LLM: 
Name of each field, its type and unique values (up to 20):
* state (string);  Values - ['NM' 'IN' 'WY' 'NH' 'MP' 'PA' 'MH' 'ID' 'AR' 'MA' 'KS' 'AS' 'ND' 'PR'
 'DE' 'FL' 'LA' 'OR' 'VT' 'PW'], ... 39 more
        


2024-03-18 01:12:39 - INFO - No filters have been set! Returning input DataFrame.


In [38]:
print(query_plan.original_query)
print(query_plan.rephrased_query)

Should I just pay the rent and wait for my refund?
advice on paying rent while awaiting a refund


In [41]:
test_res = auto_filter_fts_search(
    df=df,
    query=questions.questions[0],
    top_k=20,
    text_column="body",
    embeddings_column="embeddings",
    filter_fields=[
        'state',
    ])

2024-03-18 01:13:39 - INFO - Schema shown to LLM: 
Name of each field, its type and unique values (up to 20):
* state (string);  Values - ['NM' 'IN' 'WY' 'NH' 'MP' 'PA' 'MH' 'ID' 'AR' 'MA' 'KS' 'AS' 'ND' 'PR'
 'DE' 'FL' 'LA' 'OR' 'VT' 'PW'], ... 39 more
        


2024-03-18 01:13:41 - INFO - No filters were identified for query: Should I just pay the rent and wait for my refund?. Proceeding with the original query.
2024-03-18 01:13:42 - INFO - Full Text Search (FTS) search yielded a DataFrame with 20 rows


In [42]:
Markdown(test_res['body'].tolist()[0])

Some background - I’m planning to move out of my current apartment in Boston in June, so I’ve submitted an intent to vacate form. Also, I normally pay my rent automatically with recurring monthly payments. Well, I noticed on May 1 that I hadn’t been charged for rent, even though that transaction normally appears on my bank account on the day of, so I went to the resident portal to see what had happened. I saw that my apartment had been put “on notice,” and so my recurring payments had been cancelled. Okay, it’s not great that they didn’t tell me when I submitted my intent to vacate form that my recurring payments would be cancelled. However, at least I found out now, so this is no problem, I just submit my payment and hope that it’ll go through without having to pay any late fees.

Fast forward to May 4, I wake up and see that I received a notification that my bank account is in the negative. I panic and check to see what happened, and I see that I’ve been charged for last month’s rent twice. I shoot a message to the apartment’s customer care center but I don’t get a response until May 6:

——
Thank you for contacting the Customer Care Center.

We are unable to void any payments on your account at this time, as they are all too far along in processing. 

However, we are able to process refunds of full payments made on your account. In order for us to do so, you will need to call us at the number below during business hours so we can conference call your bank and verify the payment will not be returned. Otherwise, we will need to wait at least 14 days after the payment to process a refund. In addition, a $35.00 administrative fee would be incurred for the processing of a refund.

You can also leave your credit balance as of 05/06/2020 to be applied to future charges.

If you have any further questions or concerns, or would like to begin the process of refunding a payment, please call our number below. We are open Monday through Friday from 8:30AM EST to 8:30PM EST and Saturdays from 12:00PM EST to 5:00PM EST.
——

So, I have to pay a transaction fee to get my money back? For something that I don’t even think was my fault?... and in the meantime, I just have to live with a negative bank account balance? Is there anything I can do here?